In [4]:
#先在"大專院校校務資訊公開平台"中的"語文 學門"中的"臺灣語文細學類"網頁抓取所有學校資訊，列成表格
import requests
from tools import get_soup
import pandas as pd
from bs4 import BeautifulSoup
url = "https://udb.moe.edu.tw/udata/ISCED/%E8%AA%9E%E6%96%87/023/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87/02321"

# 發送HTTP請求
response = requests.get(url)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到表格
    table = soup.find('table')

    # 使用pandas的read_html函數直接讀取表格
    df2 = pd.read_html(str(table))[0]

    # 保留所需的欄位
    df2 = df2[['設立別', '學校類別', '學校名稱']]

    # 將資料輸出為表格形式
    print(df2.to_string(index=False))
else:
    print("無法取得網頁內容，錯誤碼：", response.status_code)


設立別 學校類別     學校名稱
 公立 一般大學   國立政治大學
 公立 一般大學   國立清華大學
 公立 一般大學   國立臺灣大學
 公立 一般大學 國立臺灣師範大學
 公立 一般大學   國立成功大學
 公立 一般大學   國立中興大學
 公立 一般大學   國立中正大學
 公立 一般大學 國立高雄師範大學
 公立 一般大學 國立彰化師範大學
 公立 一般大學   國立東華大學
 公立 一般大學   國立聯合大學
 公立 一般大學 國立臺中教育大學
 私立 一般大學     靜宜大學
 私立 一般大學     真理大學


In [6]:
# 保存成CSV文件
df2.to_csv('臺灣語文細學類學校資訊.csv', index=False, encoding='utf-8-sig')

In [33]:
#試著用Selenium自由進出網頁
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url = "https://udb.moe.edu.tw/udata/ISCED/%E8%AA%9E%E6%96%87/023/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87/02321"
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)


try:
    # 打開網頁
    driver.get(url)

    # 等待元素出現，這裡以 "國立政治大學" 的超連結為例
    university_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "國立政治大學"))
    )
    
    # 點擊 "國立政治大學" 連結
    university_link.click()
    
    # 繼續等待"臺灣語文細學類"的超連結元素
    university2_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "臺灣語文細學類"))
    )
    # 點擊 "臺灣語文細學類" 連結
    university2_link.click()
    
    # 繼續等待"國立政治大學"的超連結元素
    university3_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "國立政治大學"))
    )
    # 點擊 "國立政治大學" 連結
    university3_link.click()
    
except Exception as e:
    print(f"發生例外: {str(e)}")
    
finally:
    # 關閉瀏覽器
    driver.quit()


In [36]:
#試圖抓取"國立政治大學"中的111學年度在學學生數，利用所含文字查找
import time
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url = "https://udb.moe.edu.tw/udata/ISCED/%E8%AA%9E%E6%96%87/023/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87/02321"
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

try:
    # 打開網頁
    driver.get(url)

    # 等待 "國立政治大學" 這個連結出現
    university_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "國立政治大學"))
    )

    # 點擊 "國立政治大學" 連結
    university_link.click()
    
    response = requests.get(url, timeout=5)
    response.raise_for_status()  # 檢查網頁請求是否成功
    soup = BeautifulSoup(response.content, 'html.parser')

    datas = []
    students=soup.find_all('Student_List', class_='tab-pane fade in active')
    
    for student in students:
        number=student.find('h2').text.strip()
        
        datas.append({
            '學生數':number
        })
        
    for data in datas:
        print(f"學生數:{data['學生數']}")

    import pandas as pd
    df = pd.DataFrame(datas,columns=['學生數'])
    df

except Exception as e:
    print(f"發生例外: {str(e)}")

finally:
    # 不要在這裡關閉瀏覽器，以保留它開啟
    pass

發生例外: HTTPSConnectionPool(host='udb.moe.edu.tw', port=443): Read timed out. (read timeout=5)


In [54]:
#以"國立政治大學-臺灣語文細學類"為連結，試圖利用文字直接查找抓取"國立政治大學"中的111學年度在學學生數，列成表格
url3 = 'https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619D/0001/%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
    
response = requests.get(url3, timeout=5)
response.raise_for_status()  # 檢查網頁請求是否成功
soup = BeautifulSoup(response.content, 'html.parser')

datas = []
students=soup.find_all('Student_List', class_='tab-pane fade in active')

for student in students:
    number=student.find('h2', class_='fs-3').text.strip()

    datas.append({
        '學生數':number
    })

for data in datas:
    print(f"學生數:{data['學生數']}")

import pandas as pd
df = pd.DataFrame(datas,columns=['學生數'])
df
    

,學生數


In [55]:
#乾脆將"國立政治大學-臺灣語文細學類"所有"文字"列出來
import requests
from bs4 import BeautifulSoup

url3 = "https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619D/0001/%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87"

# 發送HTTP請求
response = requests.get(url3)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到所有的文字內容
    all_text = soup.get_text()

    # 打印結果
    print(all_text)

else:
    print("請求失敗，狀態碼：", response.status_code)













統計資訊一覽
教育部大專校院校務資訊公開平臺



















            很抱歉，網站的某些功能在JavaScript沒有啟用的狀態下無法正常使用。
        

跳到主要內容






資料處理中，請稍待…













×
提醒！


所匯出的檔案，無相關統計資料…









×
提醒！


所匯出的檔案，無相關統計資料…











Toggle navigation









                                
                                
                                首頁
                            




                            網站導覽
                        




                            計畫緣起
                        




                            連絡我們
                        





















                    :::
                



資訊介紹
資訊查詢
歷史資訊(105學年度以前)
專案輔導學校
訊息公告
相關連結












                :::
            
            網站尋覽列：
                首頁(以領域查詢)

臺灣語文學門

臺灣語文細學類

 統計資訊一覽








國立政治大學-臺灣語文細學類






學生類


教職類


校務類







































































































































                    :::
            

發現學生類、教職類、校務類內的所有資訊及其他文字在JavaScript沒有啟用的狀態下無法正常使用、爬取。
如果網站中的文字是透過JavaScript生成，而不是在HTML原始碼中靜態存在，即直接抓取文字時輸出會出現"很抱歉，網站的某些功能在JavaScript沒有啟用的狀態下無法正常使用。"，就需要使用瀏覽器自動化工具(如Selenium)，來模擬瀏覽器行為並取得生成的文字。

In [61]:
#先試試看進入"國立政治大學-學1-1.正式學籍在學學生人數-以「系(所)」統計-臺灣語文細學類"抓取表格資料以獲得111學年度在學學生數
import pandas as pd

# 發送GET請求獲得數據
url4='https://udb.moe.edu.tw/udata/DetailReport/StatStudent/TourFilter/Parameter?Parameter=%7B%22Year%22%3A%5B%22111%22%5D%2C%22AreaId%22%3A%22N%22%2C%22AreaName%22%3A%22N%22%2C%22UniversityKey%22%3A%22000012CE619D%22%2C%22UniversityId%22%3A%220001%22%2C%22UniversityList%22%3Anull%2C%22UniversityName%22%3A%22%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8%22%2C%22IscedId%22%3A%2202321%22%2C%22IscedName%22%3A%22%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87%22%2C%22EduSystemId%22%3A%22N%22%2C%22EduSystemName%22%3A%22N%22%2C%22PageSize%22%3A10%2C%22PageNumber%22%3A1%2C%22ExportType%22%3Anull%2C%22AuditStatus%22%3Anull%7D'
response = requests.get(url4)

# 檢查請求是否成功
if response.status_code == 200:
    # 從HTML中提取表格數據
    dfs = pd.read_html(response.text)

    # 假設目標表格是第一個表格，從第一個開始抓取
    target_table = dfs[0].iloc[:, 1:]  

    # 輸出表格
    print(target_table)

else:
    print("Failed to retrieve data. Status code:", response.status_code)


  設立別  學校類別    學校名稱     系所名稱     學制班別 在學學生數        
  設立別  學校類別    學校名稱     系所名稱     學制班別    小計   男   女
0  公立  一般大學  國立政治大學  台灣文學研究所  碩士班(日間)    41  20  21
1  公立  一般大學  國立政治大學  台灣文學研究所      博士班     9   4   5


In [56]:

#以"國立政治大學-學1-1.正式學籍在學學生人數-以「系(所)」統計-臺灣語文細學類"為連結，爬取所有文字並找到111學年度在學學生數的數據
url4 = "https://udb.moe.edu.tw/udata/DetailReport/StatStudent/TourFilter/Parameter?Parameter=%7B%22Year%22%3A%5B%22111%22%5D%2C%22AreaId%22%3A%22N%22%2C%22AreaName%22%3A%22N%22%2C%22UniversityKey%22%3A%22000012CE619D%22%2C%22UniversityId%22%3A%220001%22%2C%22UniversityList%22%3Anull%2C%22UniversityName%22%3A%22%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8%22%2C%22IscedId%22%3A%2202321%22%2C%22IscedName%22%3A%22%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87%22%2C%22EduSystemId%22%3A%22N%22%2C%22EduSystemName%22%3A%22N%22%2C%22PageSize%22%3A10%2C%22PageNumber%22%3A1%2C%22ExportType%22%3Anull%2C%22AuditStatus%22%3Anull%7D"

# 發送HTTP請求
response = requests.get(url4)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到所有的文字內容
    all_text = soup.get_text()

    # 打印結果
    print(all_text)

else:
    print("請求失敗，狀態碼：", response.status_code)













學1-1.正式學籍在學學生人數-以「系(所)」統計
教育部大專校院校務資訊公開平臺



















            很抱歉，網站的某些功能在JavaScript沒有啟用的狀態下無法正常使用。
        

跳到主要內容






資料處理中，請稍待…













×
提醒！


所匯出的檔案，無相關統計資料…









×
提醒！


所匯出的檔案，無相關統計資料…











Toggle navigation









                                
                                
                                首頁
                            




                            網站導覽
                        




                            計畫緣起
                        




                            連絡我們
                        





















                    :::
                



資訊介紹
資訊查詢
歷史資訊(105學年度以前)
專案輔導學校
訊息公告
相關連結












                :::
            
            網站尋覽列：
            
                            首頁(以領域查詢)

語文學門

臺灣語文細學類


                    統計資訊一覽
                

學1-1.正式學籍在學學生人數-以「系(所)」統計







國立政治大學-學1-1.正式學籍在學學生人數-以「系(所)」統計-臺灣語文細學類









查詢結果







 .xlsx
 .ods
 .csv





每頁筆數：


         

In [64]:
#若能找到111學年度在學學生數的數據，以串列形式輸出
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
# 使用 Chrome
driver = webdriver.Chrome()
driver.get(url4)

# 這裡加入適當的等待時間，等待JavaScript加載完成
# 例如，可以使用 driver.implicitly_wait(10) 等待最多10秒
# 或使用顯示等待 WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "table_id")))

# 提取表格
table = driver.find_element(By.TAG_NAME, 'table')

# 提取表格頭部
headers = [header.text.strip() for header in table.find_elements(By.TAG_NAME, 'th')]

# 提取表格數據
data_rows = []
for row in table.find_elements(By.TAG_NAME, 'tr')[1:]:
    data_rows.append([data.text.strip() for data in row.find_elements(By.TAG_NAME, 'td')])

# 關閉瀏覽器
driver.quit()

# 繼續後續的處理
num_columns = len(headers)
filtered_data_rows = [row for row in data_rows if len(row) == num_columns]

# 將數據組織成字典或其他適當的數據結構
data_dict = {}
for i, header in enumerate(headers):
    data_dict[header] = [row[i] for row in filtered_data_rows]

# 打印結果
for key, value in data_dict.items():
    print(f"{key}: {value}")


學年度: []
設立別: []
學校類別: []
學校名稱: []
系所名稱: []
學制班別: []
在學學生數: []
小計: []
男: []
女: []


In [3]:
import requests
import pandas as pd

url4 = "https://udb.moe.edu.tw/udata/DetailReport/StatStudent/TourFilter/Parameter?Parameter=%7B%22Year%22%3A%5B%22111%22%5D%2C%22AreaId%22%3A%22N%22%2C%22AreaName%22%3A%22N%22%2C%22UniversityKey%22%3A%22000012CE619D%22%2C%22UniversityId%22%3A%220001%22%2C%22UniversityList%22%3Anull%2C%22UniversityName%22%3A%22%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8%22%2C%22IscedId%22%3A%2202321%22%2C%22IscedName%22%3A%22%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87%22%2C%22EduSystemId%22%3A%22N%22%2C%22EduSystemName%22%3A%22N%22%2C%22PageSize%22%3A10%2C%22PageNumber%22%3A1%2C%22ExportType%22%3Anull%2C%22AuditStatus%22%3Anull%7D"

response = requests.get(url4)

# 檢查請求是否成功
if response.status_code == 200:
    # 從HTML中提取表格數據
    dfs = pd.read_html(response.text)

    # 假設目標表格是第一個表格，從第一個開始抓取
    target_table = dfs[0].iloc[:, 1:]

    # 輸出表格
    print("原始表格：")
    print(target_table)

    # 將表格中的數字加總
    sum_of_table = target_table.apply(pd.to_numeric, errors='coerce').sum()

    # 輸出加總結果
    print("\n每列數字加總：")
    print(sum_of_table)
    

else:
    print("Failed to retrieve data. Status code:", response.status_code)


原始表格：
  設立別  學校類別    學校名稱     系所名稱     學制班別 在學學生數        
  設立別  學校類別    學校名稱     系所名稱     學制班別    小計   男   女
0  公立  一般大學  國立政治大學  台灣文學研究所  碩士班(日間)    41  20  21
1  公立  一般大學  國立政治大學  台灣文學研究所      博士班     9   4   5

每列數字加總：
設立別    設立別      0.0
學校類別   學校類別     0.0
學校名稱   學校名稱     0.0
系所名稱   系所名稱     0.0
學制班別   學制班別     0.0
在學學生數  小計      50.0
       男       24.0
       女       26.0
dtype: float64


In [4]:
sum_of_table

設立別    設立別      0.0
學校類別   學校類別     0.0
學校名稱   學校名稱     0.0
系所名稱   系所名稱     0.0
學制班別   學制班別     0.0
在學學生數  小計      50.0
       男       24.0
       女       26.0
dtype: float64

In [16]:
text = '設立別    設立別      0.0學校類別   學校類別     0.0學校名稱   學校名稱     0.0系所名稱   系所名稱     0.0學制班別   學制班別     0.0在學學生數  小計      50.0男       24.0女       26.0dtype: float64'

# 要找的元素(國立政治大學111學年度在學學生數)
start_element = "小計"
end_element = ".0男"

# 找到起始元素的位置
start_index = text.find(start_element)

# 找到结束元素的位置
end_index = text.find(end_element)

# 切片取得夾擊的子字符串
result = text[start_index + len(start_element): end_index]

#去掉空格
result2 = "".join(result.split())

# 輸出结果
if type(result2)=="":
    print()
else:
    print(result2)

50


In [21]:
#得知"台灣語文細學類"網頁中的所有大學有哪幾間
import pandas as pd

# 給定連結
url = "https://udb.moe.edu.tw/udata/ISCED/%E8%AA%9E%E6%96%87/023/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87/02321"

# 讀取數據
dfs = pd.read_html(url)

# 獲取學校名稱的 DataFrame
school_df = dfs[0][["學校名稱"]]

# 設置學校名稱為索引
school_df.set_index("學校名稱", inplace=True)

# 輸出结果
print(school_df)


Empty DataFrame
Columns: []
Index: [國立政治大學, 國立清華大學, 國立臺灣大學, 國立臺灣師範大學, 國立成功大學, 國立中興大學, 國立中正大學, 國立高雄師範大學, 國立彰化師範大學, 國立東華大學, 國立聯合大學, 國立臺中教育大學, 靜宜大學, 真理大學]


In [115]:
#建立學生類表格
import pandas as pd

#知道了列的名稱，加入欄位並列成表格
original_columns = ["111學年度在學學生數", "110學年度畢業生數", "111學年度境外學位生數",
                    "110學年度休學人數", "110學年度退學人數", "110學年度學生實習數"]

index = ["國立政治大學", "國立清華大學", "國立臺灣大學", "國立臺灣師範大學",
         "國立成功大學", "國立中興大學", "國立中正大學", "國立高雄師範大學",
         "國立彰化師範大學", "國立東華大學", "國立聯合大學", "國立臺中教育大學",
         "靜宜大學", "真理大學"]

# 得到的數據，到時候一個一個填進去(先隨機亂數填入)
data = [
    [1000, 500, 50, 100, 20, 30],
    [1200, 600, 60, 120, 25, 35],
    [800, 400, 40, 80, 15, 25],
    [900, 450, 45, 90, 18, 28],
    [1100, 550, 55, 110, 22, 32],
    [950, 480, 48, 95, 19, 29],
    [1000, 490, 49, 98, 20, 30],
    [800, 400, 40, 80, 15, 25],
    [700, 350, 35, 70, 14, 22],
    [1200, 600, 60, 120, 25, 35],
    [1100, 550, 55, 110, 22, 32],
    [900, 450, 45, 90, 18, 28],
    [950, 480, 48, 95, 19, 29],
    [750, 380, 38, 75, 16, 26],
]

df = pd.DataFrame(data, columns=original_columns, index=index)

new_column_values = ["國立政治大學", "國立清華大學", "國立臺灣大學", "國立臺灣師範大學",
                     "國立成功大學", "國立中興大學", "國立中正大學", "國立高雄師範大學",
                     "國立彰化師範大學", "國立東華大學", "國立聯合大學", "國立臺中教育大學",
                     "靜宜大學", "真理大學"]

# 在 DataFrame 左側插入新列
df.insert(loc=0, column="學校名稱", value=new_column_values)

# 輸出结果
print(df)


              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學         1000         500            50         100   
國立清華大學      國立清華大學         1200         600            60         120   
國立臺灣大學      國立臺灣大學          800         400            40          80   
國立臺灣師範大學  國立臺灣師範大學          900         450            45          90   
國立成功大學      國立成功大學         1100         550            55         110   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [116]:
#先存成CSV檔看看是否格式正確
df.to_csv('output3.csv', index=False, encoding='utf-8-sig')

In [101]:
#抓取國立政治大學的學生類資料

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
import time


# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url = "https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619D/0001/%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87"
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

try:
    # 使用Selenium的定位方法找到所有符合條件的元素
    elements = driver.find_elements(By.CLASS_NAME, 'fs-3')

    # 使用 for 迴圈處理每個元素
    for i in range(0, len(elements)):  # 從索引0開始
        try:
            # 取得文字內容
            text = elements[i].text
            # 輸出文字內容
            print(text)
        except Exception as e:
            print(f"Error while getting text: {e}")

except NoSuchElementException as e:
    print(f"Element not found: {e}")

finally:
    # 關閉瀏覽器
    driver.quit()

學生數：50 人
畢業生數：9 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：5 人
學生數：4 人
僑生：0 人
港澳生：1 人
畢業境外生：0 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：1 人
下學期未滿1學期(修讀學分)：0 人
上學期：0 人次
下學期：1 人次
總計：15 人
上學期：3 人
下學期：5 人
寒假實習：0 人次
暑期實習：44 人次
全學期/單一學期：42 人次
學期期間實習：80 人次
全學年實習：37 人





In [102]:
#計算國立政治大學的110學年度學生實習數總數
student_internship=0+44+42+80+37
print(student_internship)

203


In [117]:
# 修改國立政治大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立政治大學", "111學年度在學學生數"] = 50
df.at["國立政治大學", "110學年度畢業生數"] = 9
df.at["國立政治大學", "111學年度境外學位生數"] = 5
df.at["國立政治大學", "110學年度休學人數"] = 15
df.at["國立政治大學", "110學年度退學人數"] = 8
df.at["國立政治大學", "110學年度學生實習數"] = 203

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學         1200         600            60         120   
國立臺灣大學      國立臺灣大學          800         400            40          80   
國立臺灣師範大學  國立臺灣師範大學          900         450            45          90   
國立成功大學      國立成功大學         1100         550            55         110   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [118]:
#建立教職類表格

original_columns = ["111學年度專任教師數","111學年度兼任教師數"]

index = ["國立政治大學", "國立清華大學", "國立臺灣大學", "國立臺灣師範大學",
         "國立成功大學", "國立中興大學", "國立中正大學", "國立高雄師範大學",
         "國立彰化師範大學", "國立東華大學", "國立聯合大學", "國立臺中教育大學",
         "靜宜大學", "真理大學"]

# 得到的數據，到時候一個一個填進去(先隨機亂數填入)
data = [
    [1000, 500],
    [1200, 600],
    [800, 400],
    [900, 450],
    [1100, 550],
    [950, 480],
    [1000, 490],
    [800, 400],
    [700, 350],
    [1200, 600],
    [1100, 5505],
    [900, 450],
    [950, 480],
    [750, 380],
]

df1 = pd.DataFrame(data, columns=original_columns, index=index)

new_column_values = ["國立政治大學", "國立清華大學", "國立臺灣大學", "國立臺灣師範大學",
                     "國立成功大學", "國立中興大學", "國立中正大學", "國立高雄師範大學",
                     "國立彰化師範大學", "國立東華大學", "國立聯合大學", "國立臺中教育大學",
                     "靜宜大學", "真理大學"]

# 在 DataFrame 左側插入新列
df1.insert(loc=0, column="學校名稱", value=new_column_values)

# 輸出结果
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學         1000          500
國立清華大學      國立清華大學         1200          600
國立臺灣大學      國立臺灣大學          800          400
國立臺灣師範大學  國立臺灣師範大學          900          450
國立成功大學      國立成功大學         1100          550
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [119]:
#先存成CSV檔看看是否格式正確
df1.to_csv('output3.csv', index=False, encoding='utf-8-sig')

In [120]:
#抓取國立政治大學的教職類資料
#(因為"教職類"按鈕點選之後網址不會更變，表示網頁中的內容是透過按鈕點選後使用JavaScript動態載入，可能需要模擬按鈕點擊事件並等待動態載入完成。
#如果按鈕點擊後的內容是透過 API 取得的，需要使用 Python 的 requests 模組來模擬 API 請求，並解析返回的 JSON 數據。)

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url = "https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619D/0001/%E5%9C%8B%E7%AB%8B%E6%94%BF%E6%B2%BB%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87"
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

try:
    # 使用等待機制等到 "教職類" 按鈕出現
    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[@data-toggle="pill" and @href="#Teacher_List"]'))
    )

    # 模擬點擊 "教職類" 按鈕
    button.click()
    
    #等待5秒讓網頁載入完成
    time.sleep(5)
    
    # 使用等待機制等到符合條件的元素出現
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'fs-3'))
    )

    # 使用 for 迴圈處理每個元素
    for element in elements:
        try:
            # 取得文字內容
            text = element.text
            # 輸出文字內容
            print(text)
        except Exception as e:
            print(f"Error while getting text: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    # 關閉瀏覽器
    driver.quit()

























教師數：6 人
教師數：4 人
教師數：1 人


In [121]:
# 修改國立政治大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立政治大學", "111學年度專任教師數"] = 6
df1.at["國立政治大學", "111學年度兼任教師數"] = 4

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學         1200          600
國立臺灣大學      國立臺灣大學          800          400
國立臺灣師範大學  國立臺灣師範大學          900          450
國立成功大學      國立成功大學         1100          550
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [124]:
#抓取國立清華大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619E/0002/%E5%9C%8B%E7%AB%8B%E6%B8%85%E8%8F%AF%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

#函式化
def search_student():
    try:
        # 使用Selenium的定位方法找到所有符合條件的元素
        elements = driver.find_elements(By.CLASS_NAME, 'fs-3')

        # 使用 for 迴圈處理每個元素
        for i in range(0, len(elements)):  # 從索引0開始
            try:
                # 取得文字內容
                text = elements[i].text
                # 輸出文字內容
                print(text)
            except Exception as e:
                print(f"Error while getting text: {e}")

    except NoSuchElementException as e:
        print(f"Element not found: {e}")

    finally:
        # 關閉瀏覽器
        driver.quit()
    
search_student()

學生數：140 人
畢業生數：17 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：14 人
學生數：2 人
僑生：2 人
港澳生：2 人
學生數：8 人
畢業境外生：1 人
上學期：3 人次
下學期：3 人次
總計：42 人
上學期：8 人
下學期：5 人
寒假實習：1 人次
暑期實習：70 人次
全學期/單一學期：77 人次
學期期間實習：84 人次
全學年實習：41 人




In [125]:
#計算國立清華大學的110學年度學生實習數總數
student_internship=1+70+77+84+41
print(student_internship)

273


In [126]:
# 修改國立清華大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立清華大學", "111學年度在學學生數"] = 140
df.at["國立清華大學", "110學年度畢業生數"] = 17
df.at["國立清華大學", "111學年度境外學位生數"] = 14
df.at["國立清華大學", "110學年度休學人數"] = 42
df.at["國立清華大學", "110學年度退學人數"] = 13
df.at["國立清華大學", "110學年度學生實習數"] = 273

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學          800         400            40          80   
國立臺灣師範大學  國立臺灣師範大學          900         450            45          90   
國立成功大學      國立成功大學         1100         550            55         110   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [128]:
#抓取國立清華大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619E/0002/%E5%9C%8B%E7%AB%8B%E6%B8%85%E8%8F%AF%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#函式化
def search_teacher():
    try:
        # 使用等待機制等到 "教職類" 按鈕出現
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[@data-toggle="pill" and @href="#Teacher_List"]'))
        )

        # 模擬點擊 "教職類" 按鈕
        button.click()

        #等待5秒讓網頁載入完成
        time.sleep(5)

        # 使用等待機制等到符合條件的元素出現
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'fs-3'))
        )

        # 使用 for 迴圈處理每個元素
        for element in elements:
            try:
                # 取得文字內容
                text = element.text
                # 輸出文字內容
                print(text)
            except Exception as e:
                print(f"Error while getting text: {e}")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # 關閉瀏覽器
        driver.quit()

search_teacher()





















教師數：16 人
教師數：10 人


In [129]:
# 修改國立清華大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立清華大學", "111學年度專任教師數"] = 16
df1.at["國立清華大學", "111學年度兼任教師數"] = 10

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學          800          400
國立臺灣師範大學  國立臺灣師範大學          900          450
國立成功大學      國立成功大學         1100          550
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [133]:
#抓取國立臺灣大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619F/0003/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：64 人
畢業生數：6 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：5 人
學生數：2 人
僑生：0 人
港澳生：3 人
學生數：0 人
畢業境外生：2 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：2 人次
下學期：5 人次
總計：15 人
上學期：1 人
下學期：1 人
寒假實習：5 人次
暑期實習：385 人次
全學期/單一學期：471 人次
學期期間實習：787 人次
全學年實習：380 人




In [134]:
#計算國立臺灣大學的110學年度學生實習數總數
student_internship=5+385+471+787+380
print(student_internship)

2028


In [135]:
# 修改國立臺灣大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立臺灣大學", "111學年度在學學生數"] = 64
df.at["國立臺灣大學", "110學年度畢業生數"] = 6
df.at["國立臺灣大學", "111學年度境外學位生數"] = 5
df.at["國立臺灣大學", "110學年度休學人數"] = 15
df.at["國立臺灣大學", "110學年度退學人數"] = 2
df.at["國立臺灣大學", "110學年度學生實習數"] = 2028

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          900         450            45          90   
國立成功大學      國立成功大學         1100         550            55         110   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [136]:
#抓取國立臺灣大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE619F/0003/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()

























教師數：7 人
教師數：2 人


In [137]:
# 修改國立臺灣大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立臺灣大學", "111學年度專任教師數"] = 7
df1.at["國立臺灣大學", "111學年度兼任教師數"] = 2

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學          900          450
國立成功大學      國立成功大學         1100          550
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [138]:
#抓取國立臺灣師範大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A0/0004/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：184 人
畢業生數：31 人
取得輔系資格：0 人
取得雙主修資格：1 人
學生數：29 人
學生數：12 人
僑生：9 人
港澳生：8 人
學生數：0 人
畢業境外生：2 人
學生數：17 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：16 人次
下學期：30 人次
上學期：5 人次
下學期：3 人次
上學期：15 人次
下學期：11 人次
總計：26 人
上學期：6 人
下學期：3 人
寒假實習：0 人次
暑期實習：403 人次
全學期/單一學期：1 人次
學期期間實習：233 人次
全學年實習：8 人





In [139]:
#計算國立臺灣師範大學的110學年度學生實習數總數
student_internship=0+403+1+233+8
print(student_internship)

645


In [140]:
# 修改國立臺灣師範大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立臺灣師範大學", "111學年度在學學生數"] = 184
df.at["國立臺灣師範大學", "110學年度畢業生數"] = 31
df.at["國立臺灣師範大學", "111學年度境外學位生數"] = 29
df.at["國立臺灣師範大學", "110學年度休學人數"] = 26
df.at["國立臺灣師範大學", "110學年度退學人數"] = 9
df.at["國立臺灣師範大學", "110學年度學生實習數"] = 645

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學         1100         550            55         110   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [141]:
#抓取國立臺灣師範大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A0/0004/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()






























教師數：12 人
教師數：5 人
教師數：2 人


In [142]:
# 修改國立臺灣師範大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立臺灣師範大學", "111學年度專任教師數"] = 12
df1.at["國立臺灣師範大學", "111學年度兼任教師數"] = 5

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學         1100          550
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [143]:
#抓取國立成功大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A1/0005/%E5%9C%8B%E7%AB%8B%E6%88%90%E5%8A%9F%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：278 人
畢業生數：38 人
取得輔系資格：2 人
取得雙主修資格：3 人
學生數：30 人
學生數：17 人
僑生：5 人
港澳生：8 人
學生數：0 人
畢業境外生：0 人
學生數：12 人
上學期至少1學期(修讀學分)以上：1 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：1 人
下學期未滿1學期(修讀學分)：0 人
上學期：4 人次
下學期：4 人次
上學期：15 人次
下學期：14 人次
上學期：20 人次
下學期：17 人次
總計：32 人
上學期：12 人
下學期：9 人
寒假實習：52 人次
暑期實習：282 人次
全學期/單一學期：347 人次
學期期間實習：965 人次
全學年實習：192 人




In [144]:
#計算國立成功大學的110學年度學生實習數總數
student_internship=52+282+347+965+192
print(student_internship)

1838


In [145]:
# 修改國立成功大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立成功大學", "111學年度在學學生數"] = 278
df.at["國立成功大學", "110學年度畢業生數"] = 38
df.at["國立成功大學", "111學年度境外學位生數"] = 30
df.at["國立成功大學", "110學年度休學人數"] = 32
df.at["國立成功大學", "110學年度退學人數"] = 21
df.at["國立成功大學", "110學年度學生實習數"] = 1838

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          950         480            48          95   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [146]:
#抓取國立成功大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A1/0005/%E5%9C%8B%E7%AB%8B%E6%88%90%E5%8A%9F%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()






























教師數：16 人
教師數：17 人


In [147]:
# 修改國立成功大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立成功大學", "111學年度專任教師數"] = 16
df1.at["國立成功大學", "111學年度兼任教師數"] = 17

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學          950          480
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [148]:
#抓取國立中興大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A2/0006/%E5%9C%8B%E7%AB%8B%E4%B8%AD%E8%88%88%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：125 人
畢業生數：11 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：12 人
學生數：9 人
僑生：1 人
港澳生：2 人
畢業境外生：2 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：0 人次
下學期：1 人次
下學期：2 人次
下學期：4 人次
總計：23 人
上學期：9 人
下學期：9 人
寒假實習：44 人次
暑期實習：212 人次
全學期/單一學期：0 人次
學期期間實習：37 人次
全學年實習：88 人







In [149]:
#計算國立中興大學的110學年度學生實習數總數
student_internship=44+212+0+37+88
print(student_internship)

381


In [150]:
# 修改國立中興大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立中興大學", "111學年度在學學生數"] = 125
df.at["國立中興大學", "110學年度畢業生數"] = 11
df.at["國立中興大學", "111學年度境外學位生數"] = 12
df.at["國立中興大學", "110學年度休學人數"] = 23
df.at["國立中興大學", "110學年度退學人數"] = 18
df.at["國立中興大學", "110學年度學生實習數"] = 381

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學         1000         490            49          98   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [151]:
#抓取國立中興大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A2/0006/%E5%9C%8B%E7%AB%8B%E4%B8%AD%E8%88%88%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()






















教師數：11 人
教師數：5 人
教師數：2 人




In [152]:
# 修改國立中興大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立中興大學", "111學年度專任教師數"] = 11
df1.at["國立中興大學", "111學年度兼任教師數"] = 5

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學         1000          490
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [153]:
#抓取國立中正大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A7/0013/%E5%9C%8B%E7%AB%8B%E4%B8%AD%E6%AD%A3%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：74 人
畢業生數：13 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：1 人
僑生：0 人
港澳生：1 人
上學期：0 人次
下學期：0 人次
總計：23 人
上學期：3 人
下學期：4 人
寒假實習：0 人次
暑期實習：158 人次
全學期/單一學期：44 人次
學期期間實習：43 人次
全學年實習：23 人




In [154]:
#計算國立中正大學的110學年度學生實習數總數
student_internship=158+44+43+23
print(student_internship)

268


In [155]:
# 修改國立中正大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立中正大學", "111學年度在學學生數"] = 74
df.at["國立中正大學", "110學年度畢業生數"] = 13
df.at["國立中正大學", "111學年度境外學位生數"] = 1
df.at["國立中正大學", "110學年度休學人數"] = 23
df.at["國立中正大學", "110學年度退學人數"] = 7
df.at["國立中正大學", "110學年度學生實習數"] = 268

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          800         400            40          80   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [156]:
#抓取國立中正大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A7/0013/%E5%9C%8B%E7%AB%8B%E4%B8%AD%E6%AD%A3%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()


















教師數：5 人
教師數：3 人


In [157]:
# 修改國立中正大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立中正大學", "111學年度專任教師數"] = 5
df1.at["國立中正大學", "111學年度兼任教師數"] = 3

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學          800          400
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [158]:
#抓取國立高雄師範大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A8/0014/%E5%9C%8B%E7%AB%8B%E9%AB%98%E9%9B%84%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：201 人
畢業生數：11 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：4 人
學生數：2 人
僑生：2 人
港澳生：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：2 人次
下學期：1 人次
上學期：7 人次
下學期：7 人次
總計：40 人
上學期：10 人
下學期：5 人
寒假實習：6 人次
暑期實習：55 人次
全學期/單一學期：41 人次
學期期間實習：98 人次
全學年實習：31 人




In [159]:
#計算國立高雄師範大學的110學年度學生實習數總數
student_internship=6+55+41+98+31
print(student_internship)

231


In [160]:
# 修改國立高雄師範大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立高雄師範大學", "111學年度在學學生數"] = 201
df.at["國立高雄師範大學", "110學年度畢業生數"] = 11
df.at["國立高雄師範大學", "111學年度境外學位生數"] = 4
df.at["國立高雄師範大學", "110學年度休學人數"] = 40
df.at["國立高雄師範大學", "110學年度退學人數"] = 15
df.at["國立高雄師範大學", "110學年度學生實習數"] = 231

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學          700         350            35          70   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [161]:
#抓取國立高雄師範大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A8/0014/%E5%9C%8B%E7%AB%8B%E9%AB%98%E9%9B%84%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()























教師數：9 人
教師數：11 人


In [162]:
# 修改國立高雄師範大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立高雄師範大學", "111學年度專任教師數"] = 9
df1.at["國立高雄師範大學", "111學年度兼任教師數"] = 11

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學          700          350
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [163]:
#抓取國立彰化師範大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A9/0015/%E5%9C%8B%E7%AB%8B%E5%BD%B0%E5%8C%96%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：27 人
畢業生數：2 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
總計：14 人
上學期：5 人
下學期：0 人
寒假實習：0 人次
暑期實習：35 人次
全學期/單一學期：35 人次
學期期間實習：26 人次
全學年實習：39 人




In [164]:
#計算國立彰化師範大學的110學年度學生實習數總數
student_internship=35+35+26+39
print(student_internship)

135


In [165]:
# 修改國立彰化師範大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立彰化師範大學", "111學年度在學學生數"] = 27
df.at["國立彰化師範大學", "110學年度畢業生數"] = 2
df.at["國立彰化師範大學", "111學年度境外學位生數"] = 0
df.at["國立彰化師範大學", "110學年度休學人數"] = 14
df.at["國立彰化師範大學", "110學年度退學人數"] = 5
df.at["國立彰化師範大學", "110學年度學生實習數"] = 135

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學         1200         600            60         120   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [166]:
#抓取國立彰化師範大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A9/0015/%E5%9C%8B%E7%AB%8B%E5%BD%B0%E5%8C%96%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()
















教師數：5 人
教師數：0 人


In [167]:
# 修改國立彰化師範大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立彰化師範大學", "111學年度專任教師數"] = 5
df1.at["國立彰化師範大學", "111學年度兼任教師數"] = 0

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學         1200          600
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [168]:
#抓取國立東華大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61AE/0020/%E5%9C%8B%E7%AB%8B%E6%9D%B1%E8%8F%AF%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：165 人
畢業生數：23 人
取得輔系資格：0 人
取得雙主修資格：1 人
學生數：7 人
學生數：0 人
僑生：2 人
港澳生：5 人
畢業境外生：2 人
學生數：15 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
下學期：0 人次
上學期：95 人次
下學期：95 人次
上學期：11 人次
下學期：11 人次
總計：10 人
上學期：7 人
下學期：5 人
寒假實習：24 人次
暑期實習：88 人次
全學期/單一學期：46 人次
學期期間實習：162 人次
全學年實習：26 人




In [169]:
#計算國立東華大學的110學年度學生實習數總數
student_internship=24+88+46+162+26
print(student_internship)

346


In [170]:
# 修改國立東華大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立東華大學", "111學年度在學學生數"] = 165
df.at["國立東華大學", "110學年度畢業生數"] = 23
df.at["國立東華大學", "111學年度境外學位生數"] = 7
df.at["國立東華大學", "110學年度休學人數"] = 10
df.at["國立東華大學", "110學年度退學人數"] = 12
df.at["國立東華大學", "110學年度學生實習數"] = 346

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學          165          23             7          10   
國立聯合大學      國立聯合大學         1100         550            55         110   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [171]:
#抓取國立東華大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61AE/0020/%E5%9C%8B%E7%AB%8B%E6%9D%B1%E8%8F%AF%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()




























教師數：8 人
教師數：8 人


In [172]:
# 修改國立東華大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立東華大學", "111學年度專任教師數"] = 8
df1.at["國立東華大學", "111學年度兼任教師數"] = 8

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學            8            8
國立聯合大學      國立聯合大學         1100         5505
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [173]:
#抓取國立聯合大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61B4/0032/%E5%9C%8B%E7%AB%8B%E8%81%AF%E5%90%88%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：197 人
畢業生數：48 人
取得輔系資格：0 人
取得雙主修資格：1 人
學生數：4 人
僑生：4 人
港澳生：0 人
學生數：0 人
畢業境外生：2 人
學生數：2 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
上學期：0 人次
下學期：0 人次
上學期：6 人次
下學期：6 人次
上學期：1 人次
下學期：0 人次
總計：27 人
上學期：9 人
下學期：5 人
寒假實習：1 人次
暑期實習：36 人次
全學期/單一學期：123 人次
學期期間實習：7 人次
全學年實習：32 人




In [174]:
#計算國立聯合大學的110學年度學生實習數總數
student_internship=1+36+123+7+32
print(student_internship)

199


In [175]:
# 修改國立聯合大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立聯合大學", "111學年度在學學生數"] = 197
df.at["國立聯合大學", "110學年度畢業生數"] = 48
df.at["國立聯合大學", "111學年度境外學位生數"] = 4
df.at["國立聯合大學", "110學年度休學人數"] = 27
df.at["國立聯合大學", "110學年度退學人數"] = 14
df.at["國立聯合大學", "110學年度學生實習數"] = 199

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學          165          23             7          10   
國立聯合大學      國立聯合大學          197          48             4          27   
國立臺中教育大學  國立臺中教育大學          900         450            45          90   
靜宜大學          靜宜大學          950         480        

In [176]:
#抓取國立聯合大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61B4/0032/%E5%9C%8B%E7%AB%8B%E8%81%AF%E5%90%88%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()



























教師數：14 人
教師數：6 人


In [177]:
# 修改國立聯合大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立聯合大學", "111學年度專任教師數"] = 14
df1.at["國立聯合大學", "111學年度兼任教師數"] = 6

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學            8            8
國立聯合大學      國立聯合大學           14            6
國立臺中教育大學  國立臺中教育大學          900          450
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [178]:
#抓取國立臺中教育大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61B8/0039/%E5%9C%8B%E7%AB%8B%E8%87%BA%E4%B8%AD%E6%95%99%E8%82%B2%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：204 人
畢業生數：38 人
取得輔系資格：0 人
取得雙主修資格：0 人
學生數：0 人
僑生：0 人
港澳生：0 人
學生數：12 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：5 人次
下學期：5 人次
上學期：6 人次
下學期：7 人次
上學期：0 人次
下學期：0 人次
總計：21 人
上學期：13 人
下學期：4 人
寒假實習：0 人次
暑期實習：154 人次
全學期/單一學期：16 人次
學期期間實習：97 人次
全學年實習：17 人




In [180]:
#計算國立臺中教育大學的110學年度學生實習數總數
student_internship=154+16+97+17
print(student_internship)

284


In [181]:
# 修改國立臺中教育大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["國立臺中教育大學", "111學年度在學學生數"] = 204
df.at["國立臺中教育大學", "110學年度畢業生數"] = 38
df.at["國立臺中教育大學", "111學年度境外學位生數"] = 0
df.at["國立臺中教育大學", "110學年度休學人數"] = 21
df.at["國立臺中教育大學", "110學年度退學人數"] = 17
df.at["國立臺中教育大學", "110學年度學生實習數"] = 284

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學          165          23             7          10   
國立聯合大學      國立聯合大學          197          48             4          27   
國立臺中教育大學  國立臺中教育大學          204          38             0          21   
靜宜大學          靜宜大學          950         480        

In [182]:
#抓取國立臺中教育大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61B8/0039/%E5%9C%8B%E7%AB%8B%E8%87%BA%E4%B8%AD%E6%95%99%E8%82%B2%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()



























教師數：9 人
教師數：2 人


In [183]:
# 修改國立臺中教育大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["國立臺中教育大學", "111學年度專任教師數"] = 9
df1.at["國立臺中教育大學", "111學年度兼任教師數"] = 2

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學            8            8
國立聯合大學      國立聯合大學           14            6
國立臺中教育大學  國立臺中教育大學            9            2
靜宜大學          靜宜大學          950          480
真理大學          真理大學          750          380


In [184]:
#抓取靜宜大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61C4/1008/%E9%9D%9C%E5%AE%9C%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：239 人
畢業生數：55 人
取得輔系資格：2 人
取得雙主修資格：30 人
學生數：1 人
學生數：1 人
僑生：0 人
港澳生：0 人
學生數：0 人
畢業境外生：2 人
學生數：10 人
上學期至少1學期(修讀學分)以上：0 人
上學期未滿1學期(修讀學分)：0 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：0 人次
下學期：0 人次
上學期：4 人次
下學期：4 人次
上學期：196 人次
下學期：174 人次
總計：14 人
上學期：15 人
下學期：4 人
寒假實習：12 人次
暑期實習：301 人次
全學期/單一學期：167 人次
學期期間實習：346 人次
全學年實習：16 人




In [185]:
#計算靜宜大學的110學年度學生實習數總數
student_internship=12+301+167+346+16
print(student_internship)

842


In [186]:
# 修改靜宜大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["靜宜大學", "111學年度在學學生數"] = 239
df.at["靜宜大學", "110學年度畢業生數"] = 55
df.at["靜宜大學", "111學年度境外學位生數"] = 1
df.at["靜宜大學", "110學年度休學人數"] = 14
df.at["靜宜大學", "110學年度退學人數"] = 19
df.at["靜宜大學", "110學年度學生實習數"] = 842

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學          165          23             7          10   
國立聯合大學      國立聯合大學          197          48             4          27   
國立臺中教育大學  國立臺中教育大學          204          38             0          21   
靜宜大學          靜宜大學          239          55        

In [187]:
#抓取靜宜大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61C4/1008/%E9%9D%9C%E5%AE%9C%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()






























教師數：9 人
教師數：6 人


In [188]:
# 修改靜宜大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["靜宜大學", "111學年度專任教師數"] = 9
df1.at["靜宜大學", "111學年度兼任教師數"] = 6

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學            8            8
國立聯合大學      國立聯合大學           14            6
國立臺中教育大學  國立臺中教育大學            9            2
靜宜大學          靜宜大學            9            6
真理大學          真理大學          750          380


In [189]:
#抓取真理大學的學生類資料

# 設定 WebDriver 的路徑，這裡以 Chrome 為例
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61D0/1021/%E7%9C%9F%E7%90%86%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待10秒讓網站載入完成
time.sleep(10)

search_student()

學生數：125 人
畢業生數：21 人
取得輔系資格：1 人
取得雙主修資格：0 人
學生數：6 人
學生數：5 人
僑生：0 人
港澳生：1 人
畢業境外生：0 人
學生數：12 人
下學期至少1學期(修讀學分)以上：0 人
下學期未滿1學期(修讀學分)：0 人
上學期：1 人次
下學期：1 人次
上學期：2 人次
下學期：1 人次
上學期：0 人次
下學期：0 人次
總計：9 人
上學期：10 人
下學期：7 人
寒假實習：4 人次
暑期實習：145 人次
全學期/單一學期：30 人次
學期期間實習：190 人次
全學年實習：0 人




In [190]:
#計算真理大學的110學年度學生實習數總數
student_internship=4+145+30+190
print(student_internship)

369


In [191]:
# 修改真理大學的111學年度在學學生數、110學年度畢業生數、111學年度境外學位生數、110學年度休學人數、110學年度退學人數、110學年度學生實習數
#"111學年度在學學生數"是第1條，"110學年度畢業生數"是第2條，"111學年度境外學位生數"是第5條，"110學年度休學人數"是倒數第8條，
#"110學年度退學人數"是倒數第7與第6條加總，"110學年度學生實習數"是倒數第5條到最後1條的總和
df.at["真理大學", "111學年度在學學生數"] = 125
df.at["真理大學", "110學年度畢業生數"] = 21
df.at["真理大學", "111學年度境外學位生數"] = 6
df.at["真理大學", "110學年度休學人數"] = 9
df.at["真理大學", "110學年度退學人數"] = 17
df.at["真理大學", "110學年度學生實習數"] = 369

# 輸出修改後的表格
print(df)

              學校名稱  111學年度在學學生數  110學年度畢業生數  111學年度境外學位生數  110學年度休學人數  \
國立政治大學      國立政治大學           50           9             5          15   
國立清華大學      國立清華大學          140          17            14          42   
國立臺灣大學      國立臺灣大學           64           6             5          15   
國立臺灣師範大學  國立臺灣師範大學          184          31            29          26   
國立成功大學      國立成功大學          278          38            30          32   
國立中興大學      國立中興大學          125          11            12          23   
國立中正大學      國立中正大學           74          13             1          23   
國立高雄師範大學  國立高雄師範大學          201          11             4          40   
國立彰化師範大學  國立彰化師範大學           27           2             0          14   
國立東華大學      國立東華大學          165          23             7          10   
國立聯合大學      國立聯合大學          197          48             4          27   
國立臺中教育大學  國立臺中教育大學          204          38             0          21   
靜宜大學          靜宜大學          239          55        

In [192]:
#抓取真理大學的教職類資料

path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61D0/1021/%E7%9C%9F%E7%90%86%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

search_teacher()



























教師數：7 人
教師數：5 人


In [193]:
# 修改真理大學的111學年度專任教師數、111學年度兼任教師數
#"111學年度專任教師數"是第1條，"111學年度兼任教師數"是第2條
df1.at["真理大學", "111學年度專任教師數"] = 7
df1.at["真理大學", "111學年度兼任教師數"] = 5

# 輸出修改後的表格
print(df1)

              學校名稱  111學年度專任教師數  111學年度兼任教師數
國立政治大學      國立政治大學            6            4
國立清華大學      國立清華大學           16           10
國立臺灣大學      國立臺灣大學            7            2
國立臺灣師範大學  國立臺灣師範大學           12            5
國立成功大學      國立成功大學           16           17
國立中興大學      國立中興大學           11            5
國立中正大學      國立中正大學            5            3
國立高雄師範大學  國立高雄師範大學            9           11
國立彰化師範大學  國立彰化師範大學            5            0
國立東華大學      國立東華大學            8            8
國立聯合大學      國立聯合大學           14            6
國立臺中教育大學  國立臺中教育大學            9            2
靜宜大學          靜宜大學            9            6
真理大學          真理大學            7            5


In [194]:
# 保存成CSV文件(學生類)
df.to_csv('臺灣語文細學類大專院校學生類人數統計.csv', index=False, encoding='utf-8-sig')

In [195]:
# 保存成CSV文件(教職類)
df1.to_csv('臺灣語文細學類大專院校教職類人數統計.csv', index=False, encoding='utf-8-sig')

In [215]:
#抓取國立臺灣師範大學111學年度在學學生數人數統計資料並列成表格。

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
path = r'C:\Users\Evan\Downloads\anaconda\chromedriver-win64\chromedriver.exe'
url='https://udb.moe.edu.tw/udata/StatCardList/ISCED/000012CE61A0/0004/%E5%9C%8B%E7%AB%8B%E8%87%BA%E7%81%A3%E5%B8%AB%E7%AF%84%E5%A4%A7%E5%AD%B8/02321/%E8%87%BA%E7%81%A3%E8%AA%9E%E6%96%87'
service=Service(executable_path=path)
service
# 使用 Chrome 開啟瀏覽器
driver=webdriver.Chrome(service=service)

# 前往目標網站
driver.get(url)

#等待網頁加載
time.sleep(3)

# 點選 "看更多" 按鈕
more_button = driver.find_element(By.XPATH, '//a[@class="btn btn-primary"]')
more_button.click()
time.sleep(5)

# 等待新網頁加載，最多等待10秒
WebDriverWait(driver, 10).until(
    EC.url_changes(url)
)

# 取得新網頁的 URL
new_page_url = driver.current_url

# 發送 GET 請求
response = requests.get(new_page_url)

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 找到表格
table = soup.find('table')

# 找到表格的標題行
header_row = table.find('tr')

# 提取表格的欄位名稱
columns = [col.text.strip() for col in header_row.find_all('th')]

# 顯示欄位名稱
print("\t".join(columns))

# 找到表格的所有資料行
data_rows = table.find_all('tr')[1:]

# 逐行提取資料
for row in data_rows:
    # 提取每一行的所有欄位
    columns_data = [col.text.strip() for col in row.find_all('td')]
    
    # 顯示每一行的資料
    print("\t".join(columns_data))

# 關閉瀏覽器
driver.quit()

學年度	設立別	學校類別	學校名稱	系所名稱	學制班別	在學學生數

111	公立	一般大學	國立臺灣師範大學	臺灣語文學系	學士班(日間)	105	40	65
111	公立	一般大學	國立臺灣師範大學	臺灣語文學系	碩士班(日間)	52	27	25
111	公立	一般大學	國立臺灣師範大學	臺灣語文學系	博士班	27	12	15


In [218]:
# 保存成CSV文件(國立臺灣師範大學111學年度在學學生數人數統計資料)
import csv
with open('國立臺灣師範大學-學1-1.正式學籍在學學生人數-以「系(所)」統計-臺灣語文細學類.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(columns)

    # 逐行提取資料，並寫入 CSV 文件
    for row in data_rows:
        columns_data = [col.text.strip() for col in row.find_all('td')]
        csv_writer.writerow(columns_data)
